In [99]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
%matplotlib inline
import json

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

### Buisiness Understanding

Q1. Which groups of people are most responsive to each type of offer?  
Q2. How to present each type of offer?

### Data Understanding

**Portfolio**: Offers sent during 30-day test period (10 offers x 6 fields)
- reward: (numeric) money awarded for the amount spent
- channels: (list) web, email, mobile, social
- difficulty: (numeric) money required to be spent to receive reward
- duration: (numeric) time for offer to be open, in days
- offer_type: (string) bogo, discount, informational
- id: (string/hash)

In [3]:
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [4]:
portfolio.shape

(10, 6)

ex)
0 row의 경우, 10 달러의 리워드가 있으며 채널은 email, mobile, social을 통해 고객에게 전달됨.   
리워드를 받기 위해선 10 달러를 써야 하며, 사용 가능 기간은 7일임.  
프로모션의 형태는 bogo(하나사면 하나 더 줌)이며 해당 프로모션의 id는 ae264어쩌구저쩌구임.

**Profile**: Rewards program users (17000 users x 5 fields)
- gender: (categorical) M, F, O, or null
- age: (numeric) missing value encoded as 118
- id: (string/hash)
- became_member_on: (date) format YYYYMMDD
- income: (numeric)

In [5]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [6]:
profile.shape

(17000, 5)

ex) 0 row의 경우, 성별은 None임(그놈의 PC충들). 나이는 무려 118살이네. 고객 id는 위와 같음.  
2017년2월12일에 고객이 되었으며, 수익은 없는듯함.

**Transcript**: Event log (306648 events x 4 fields)
- person: (string/hash)
- event: (string) offer received, offer viewed, transaction, offer completed
- value: (dictionary) different values depending on event type
- offer id: (string/hash) not associated with any "transaction"
- amount: (numeric) money spent in "transaction"
- reward: (numeric) money gained from "offer completed"
- time: (numeric) hours after start of test

In [7]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [8]:
transcript.shape

(306534, 4)

ex) 0 row의 경우, 고객 id는 78afa어쩌구저쩌구이며(profile 셋에 있음), 오퍼를 받은 것이 확인됨.  
오퍼의 종류는 9b98b어쩌구저쩌구이고(portfolio 셋에 있음), 테스트 시작하자마자 오퍼를 받음.

### Data Preparing

In [17]:
transcript['event'].unique()

array(['offer received', 'offer viewed', 'transaction', 'offer completed'],
      dtype=object)

In [16]:
transcript[transcript['event']!='offer received']

,person,event,value,time
12650,389bc3fa690240e798340f5a15918d5c,offer viewed,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},0
12651,d1ede868e29245ea91818a903fec04c6,offer viewed,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'},0
12652,102e9454054946fda62242d2e176fdce,offer viewed,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0
12653,02c083884c7d45b39cc68e1314fec56c,offer viewed,{'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'},0
12654,02c083884c7d45b39cc68e1314fec56c,transaction,{'amount': 0.8300000000000001},0
...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},714
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714
306531,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,{'amount': 3.5300000000000002},714


In [25]:
set(transcript[transcript['event']=='offer completed'].person.unique()) == set(transcript[transcript['event']=='transaction'].person.unique())

False

In [33]:
transcript['person'].nunique()

17000

In [31]:
transcript[transcript['event']=='offer received'].person.nunique()

16994

In [32]:
transcript[transcript['event']=='offer viewed'].person.nunique()

16834

In [26]:
transcript[transcript['event']=='transaction'].person.nunique()

16578

In [27]:
transcript[transcript['event']=='offer completed'].person.nunique()

12774

In [35]:
transcript[transcript['event']=='offer received'].value

0         {'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1         {'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2         {'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3         {'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4         {'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}
                                ...                       
257882    {'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'}
257883    {'offer id': '2906b810c7d4411798c6938adc9daaa5'}
257884    {'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'}
257885    {'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'}
257886    {'offer id': '3f207df678b143eea3cee63160fa8bed'}
Name: value, Length: 76277, dtype: object

In [50]:
transcript[transcript['event']=='offer viewed'].value

12650     {'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'}
12651     {'offer id': '5a8bc65990b245e5a138643cd4eb9837'}
12652     {'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}
12653     {'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'}
12655     {'offer id': '5a8bc65990b245e5a138643cd4eb9837'}
                                ...                       
306441    {'offer id': '5a8bc65990b245e5a138643cd4eb9837'}
306450    {'offer id': '3f207df678b143eea3cee63160fa8bed'}
306483    {'offer id': '5a8bc65990b245e5a138643cd4eb9837'}
306490    {'offer id': '3f207df678b143eea3cee63160fa8bed'}
306507    {'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
Name: value, Length: 57725, dtype: object

In [51]:
transcript[transcript['event']=='transaction'].value

12654     {'amount': 0.8300000000000001}
12657                  {'amount': 34.56}
12659                  {'amount': 13.23}
12670                  {'amount': 19.51}
12671                  {'amount': 18.97}
                       ...              
306529    {'amount': 1.5899999999999999}
306530                  {'amount': 9.53}
306531                  {'amount': 3.61}
306532    {'amount': 3.5300000000000002}
306533                  {'amount': 4.05}
Name: value, Length: 138953, dtype: object

In [97]:
transcript[transcript['event']=='offer completed'].value.iloc[10]

AttributeError: 'DataFrame' object has no attribute 'value'

In [55]:
pd.DataFrame.from_dict(transcript['value'])

,value
0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}
...,...
306529,{'amount': 1.5899999999999999}
306530,{'amount': 9.53}
306531,{'amount': 3.61}
306532,{'amount': 3.5300000000000002}


In [58]:
transcript = pd.concat([transcript.drop(['value'], axis=1), 
                        transcript.value.apply(pd.Series)], axis=1)

In [106]:
transcript.value.tolist()

[{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},
 {'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},
 {'offer id': '2906b810c7d4411798c6938adc9daaa5'},
 {'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},
 {'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},
 {'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},
 {'offer id': '2298d6c36e964ae4a3e7e9706d1fb8c2'},
 {'offer id': '3f207df678b143eea3cee63160fa8bed'},
 {'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},
 {'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},
 {'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},
 {'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},
 {'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'},
 {'offer id': '3f207df678b143eea3cee63160fa8bed'},
 {'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},
 {'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},
 {'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},
 {'offer id': '2906b810c7d4411798c6938adc9daaa5'},
 {'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},
 {'offer id': 'ae264e3637204a6f

In [111]:
pd.DataFrame(transcript['value'].tolist()).offer_id.nunique()

8

In [112]:
pd.DataFrame(transcript['value'].tolist())['offer id'].nunique()

10

In [66]:
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [87]:
extract_type = lambda x: portfolio[portfolio['id']==x]['offer_type'][0]

In [94]:
transcript['offer id'].isnull().sum()

172532